# Neural Networks

## Overview
In this lesson we will see how machine learning techniques can be successfully applied to solve financial problems. We will first do a quick tour on the theory behind neural networks and then we will see an example and two practical applications regarding regression and classification issues. 

**Disclaimer**: this lecture just scratches the surface of the machine learning topic which has seen a huge development in the latest years leading to thousands of applications in many different fields.

## Neural network Definition
Artificial Neural Networks (ANN or simply NN) are information processing models that are developed by inspiring from the working principles of human brain. Their most essential property is the ability of learning from sample sets. 

The basic unit of ANN architecture are neurons which are internally in connection with other neurons. 

![Model of an artificial neuron.](neuron.jpeg)

A neuron consists of weights ($w_i$) and real numbers ($x_i$). All inputs injected into a neuron are individually weighted, added together and passed into the activation function which produce the neuron output. There are many different types of activation function but one of the simplest is the *step function* which returns just 0 or 1 according to the input value (another is the *sigmoid* which can be thought of as the continuous version of the step function). 

![Step function.](step_function.png)
![Sigmoid function.](sigmoid.png)


## Training of a neuron

When teaching children how to recognize a bus, we just tell them, showing an example: “This is a bus. That is not a bus.” until they learn the concept of what a bus is. 
Furthermore, if the child sees new objects that she hasn’t seen before, we could expect her to recognize correctly whether the new object is a bus or not.

This is exactly the idea behind neurons.
Similarly, inputs from a *training* set are presented to the neuron one after the other together with the correct output and the neuron weights are modified accordingly.

When an entire pass through all of the input training vectors is completed the neuron has learnt ! 

At this time, if an input vector $\vec{P}$ (already in the training set) is given to the neuron, it will output the correct value. If $\vec{P}$ is not in the training set, the network will respond with an output similar to other training vectors close to $\vec{P}$.

Unfortunately using just a neuron is not too useful since it is not possible to solve
the interesting problems we would like to face with just that simple architecture. The next step is then to put together more neurons in *layers*.

### Multi-layered neural networks

![A multi-layered neural network.](multilayer.jpeg)

Each input from the *input layer* is fed up to each node in the next hidden layer, and from there to each node on the output layer. We should note that there can be any number of nodes per layer and there are usually multiple hidden layers to pass through before ultimately reaching the output layer.







But to train this network we need a learning algorithm which should be able to tune not only the weights between the output layer and the hidden layer but also the weights between the hidden layer and the input layer. 

### Back propagation

In order to tune the weights at each layer at every iteration we should know what the output would be at each node. But this is not possbile since the only value we know is the correct output at the last node (the final output of the NN which can be compared to our truth of the training output).
The method that was suggested to overcome the issue was to take the errors at the output layer (last node) and proportionally propagate it backwards to all the hidden layer.

So, what it's going to happen is:

* present a training sample to the neural network (initialized with random weights);
* compute the output received by calculating activations of each layer and thus calculate the error as the difference between the NN output and the training sample expected result;
* having calculated the error, readjust the weights such that the error (difference) decreases;
* continue the process for all training samples several times until the weights are not changing too much (a.k.a. the process converged).

The NN error is computed by the *loss function* (usually it is either the mean squared error or the mean absolute error) and an *optimization function* is then 
used to choose the appropriate weight values in order to reduce the loss function value (we will use *Adam* as optimizator in the following but there are more).








### Neural Network Design

There is no rule to guide developer into the design of a neural network in terms of number of layers and neuron per layer. The most common strategy is a trail and error one where you finally pick up the solution giving the best accuracy. In general a larger number of nodes is better to catch highly structured data with a lot of feature although it may require larger training sample to work correctly.

A common mistake to avoid is to *overtrain* a NN. Overtraining is what happens when the NN learns too well the training sample but its performance degrade substantially in an independent testing sample. 

So usually it is required to split the available sample in two parts training and testing (e.g. 80% and 20%) and to use the former to perform the training and the latter to cross-check the performance. **Usually performance are *measured* with the mean square error computed between the truth of the sample and the NN predictions.**

Anyway as a rule of thumb a NN with just one hidden layer with a number of neurons averaging the inputs and outputs is sufficient in most cases. In the following we will use more complex networks just for illustration, no strong attempt in optimizing the layout has been done though.

## Practical Examples

Below it will be illustrated few practical applications of neural network trainings in python. 
Various modules are available to develop neural network in ```python```, we will
use ```Keras``` a relatively high level library which in turn use ```TensorFlow``` a very famous machine learning tool developed by Google.

In the attempt of keeping things as simple as possible I have added another layer above ```Keras```, ```FinNN``` so that you can try 
to design some NN without caring too much about the many details and parameters that are involved in the process.

### Function approximation 

As a first practical example let's try to design an ANN which is capable of learning the functional form underlying a set of data.

Let's generate a sample with $x$ (input), $f(x)$ (truth) pairs where $f(x) = x^3 +2$ and let's start to code the NN structure. 

We start by importing the necessary modules.
Then we generate the training sample (i.e. the $x$, $f(x)$ pairs) and apply a simple transformation on the sample in order to have all the inputs and outputs in the $[0, 1]$ range. This is usually done to provide the NN with *normalized* data, infact the NN can be fooled by large or very small numbers giving unstable results.

In [10]:
from finnn import FinNN
from numpy import arange, array

# define the dataset
x = array([i for i in arange(-2, 2, 0.005)])

y = array([i**3+2 for i in x])
print("Distribution of original data ", x.min(), x.max(), y.min(), y.max())

trainer = FinNN()
trainer.setData(x, y)

trainer.normalize()

# here you should see that x and y are between 0 and 1
print("The same data after the normalization ", trainer.x.min(), 
      trainer.x.max(), trainer.y.min(), trainer.y.max())

Distribution of original data  -2.0 1.9949999999999148 -6.0 9.940149874998983
The same data after the normalization  0.0 1.0 0.0 1.0


Next we can define the structure of the neural network. There is no predefined rule to decide the number of layers and nodes you need to go by trial and error. Here the problem is quite simple so there is no need to use a complecated NN. 

In the end I have decided to use two layers with 10 nodes each and a *sigmoidal* activation function. The input_dim parameter has to be set to 1 since we have just one single input, the $x$ value. 

In [11]:
# design the neural network model
trainer.addInputLayer(1, 15, 'tanh')
trainer.addHiddenLayer(5, 'tanh')
trainer.addOutputLayer(1)

# define the loss function (mean squared error) and optimization algorithm (Adam)
trainer.compileModel('mse', 'adam')

# fit the model on the training dataset
# using 500 epochs, a batch_size of 10
trainer.fit(800, 100, verbose=1)

# make predictions for the input data
trainer.fullPrediction()

# invert the previous transformation to get back the real data and not the normalized one
trainer.reverseNormalization()

Epoch 1/800
799/799 [==============================] - 0s 127us/step - loss: 0.0919
Epoch 2/800
799/799 [==============================] - 0s 20us/step - loss: 0.0366
Epoch 3/800
799/799 [==============================] - 0s 16us/step - loss: 0.0183
Epoch 4/800
799/799 [==============================] - 0s 22us/step - loss: 0.0168
Epoch 5/800
799/799 [==============================] - 0s 25us/step - loss: 0.0156
Epoch 6/800
799/799 [==============================] - 0s 24us/step - loss: 0.0131
Epoch 7/800
799/799 [==============================] - 0s 32us/step - loss: 0.0116
Epoch 8/800
799/799 [==============================] - 0s 28us/step - loss: 0.0111
Epoch 9/800
799/799 [==============================] - 0s 22us/step - loss: 0.0106
Epoch 10/800
799/799 [==============================] - 0s 21us/step - loss: 0.0102
Epoch 11/800
799/799 [==============================] - 0s 27us/step - loss: 0.0100
Epoch 12/800
799/799 [==============================] - 0s 29us/step - loss: 0.0098


799/799 [==============================] - 0s 23us/step - loss: 0.0046
Epoch 99/800
799/799 [==============================] - 0s 16us/step - loss: 0.0046
Epoch 100/800
799/799 [==============================] - 0s 18us/step - loss: 0.0046
Epoch 101/800
799/799 [==============================] - 0s 16us/step - loss: 0.0045
Epoch 102/800
799/799 [==============================] - 0s 20us/step - loss: 0.0045
Epoch 103/800
799/799 [==============================] - 0s 24us/step - loss: 0.0045
Epoch 104/800
799/799 [==============================] - 0s 23us/step - loss: 0.0044
Epoch 105/800
799/799 [==============================] - 0s 22us/step - loss: 0.0044
Epoch 106/800
799/799 [==============================] - 0s 24us/step - loss: 0.0044
Epoch 107/800
799/799 [==============================] - 0s 22us/step - loss: 0.0043
Epoch 108/800
799/799 [==============================] - ETA: 0s - loss: 0.003 - 0s 20us/step - loss: 0.0043
Epoch 109/800
799/799 [==============================] -

799/799 [==============================] - 0s 24us/step - loss: 0.0021
Epoch 194/800
799/799 [==============================] - 0s 25us/step - loss: 0.0021
Epoch 195/800
799/799 [==============================] - 0s 23us/step - loss: 0.0021
Epoch 196/800
799/799 [==============================] - 0s 24us/step - loss: 0.0020
Epoch 197/800
799/799 [==============================] - 0s 23us/step - loss: 0.0020
Epoch 198/800
799/799 [==============================] - 0s 22us/step - loss: 0.0020
Epoch 199/800
799/799 [==============================] - 0s 21us/step - loss: 0.0020
Epoch 200/800
799/799 [==============================] - 0s 18us/step - loss: 0.0019
Epoch 201/800
799/799 [==============================] - 0s 12us/step - loss: 0.0020
Epoch 202/800
799/799 [==============================] - 0s 15us/step - loss: 0.0019
Epoch 203/800
799/799 [==============================] - 0s 21us/step - loss: 0.0019
Epoch 204/800
799/799 [==============================] - 0s 23us/step - loss: 0

Epoch 289/800
799/799 [==============================] - 0s 13us/step - loss: 0.0010
Epoch 290/800
799/799 [==============================] - 0s 27us/step - loss: 0.0010
Epoch 291/800
799/799 [==============================] - 0s 20us/step - loss: 0.0010
Epoch 292/800
799/799 [==============================] - 0s 20us/step - loss: 9.9037e-04
Epoch 293/800
799/799 [==============================] - 0s 31us/step - loss: 9.9039e-04
Epoch 294/800
799/799 [==============================] - 0s 20us/step - loss: 9.7517e-04
Epoch 295/800
799/799 [==============================] - 0s 18us/step - loss: 9.8004e-04
Epoch 296/800
799/799 [==============================] - 0s 18us/step - loss: 0.0010
Epoch 297/800
799/799 [==============================] - 0s 17us/step - loss: 0.0010
Epoch 298/800
799/799 [==============================] - 0s 17us/step - loss: 9.6979e-04
Epoch 299/800
799/799 [==============================] - 0s 23us/step - loss: 9.5715e-04
Epoch 300/800
799/799 [==================

799/799 [==============================] - 0s 17us/step - loss: 6.0318e-04
Epoch 381/800
799/799 [==============================] - 0s 17us/step - loss: 6.0122e-04
Epoch 382/800
799/799 [==============================] - 0s 15us/step - loss: 6.0949e-04
Epoch 383/800
799/799 [==============================] - 0s 14us/step - loss: 6.0684e-04
Epoch 384/800
799/799 [==============================] - 0s 15us/step - loss: 5.8714e-04
Epoch 385/800
799/799 [==============================] - 0s 15us/step - loss: 5.9114e-04
Epoch 386/800
799/799 [==============================] - 0s 17us/step - loss: 5.8635e-04
Epoch 387/800
799/799 [==============================] - 0s 16us/step - loss: 5.8093e-04
Epoch 388/800
799/799 [==============================] - 0s 20us/step - loss: 5.8364e-04
Epoch 389/800
799/799 [==============================] - 0s 16us/step - loss: 5.9049e-04
Epoch 390/800
799/799 [==============================] - 0s 14us/step - loss: 5.9395e-04
Epoch 391/800
799/799 [============

799/799 [==============================] - 0s 24us/step - loss: 3.2793e-04
Epoch 472/800
799/799 [==============================] - 0s 17us/step - loss: 3.2353e-04
Epoch 473/800
799/799 [==============================] - 0s 21us/step - loss: 3.2151e-04
Epoch 474/800
799/799 [==============================] - 0s 18us/step - loss: 3.2214e-04
Epoch 475/800
799/799 [==============================] - 0s 20us/step - loss: 3.1454e-04
Epoch 476/800
799/799 [==============================] - 0s 21us/step - loss: 3.1058e-04
Epoch 477/800
799/799 [==============================] - 0s 26us/step - loss: 3.1368e-04
Epoch 478/800
799/799 [==============================] - 0s 19us/step - loss: 3.0544e-04
Epoch 479/800
799/799 [==============================] - 0s 17us/step - loss: 3.0613e-04
Epoch 480/800
799/799 [==============================] - 0s 20us/step - loss: 3.0341e-04
Epoch 481/800
799/799 [==============================] - 0s 21us/step - loss: 3.0173e-04
Epoch 482/800
799/799 [============

799/799 [==============================] - 0s 20us/step - loss: 1.6088e-04
Epoch 563/800
799/799 [==============================] - 0s 19us/step - loss: 1.6021e-04
Epoch 564/800
799/799 [==============================] - 0s 19us/step - loss: 1.6753e-04
Epoch 565/800
799/799 [==============================] - 0s 20us/step - loss: 1.7085e-04
Epoch 566/800
799/799 [==============================] - 0s 19us/step - loss: 1.7164e-04
Epoch 567/800
799/799 [==============================] - 0s 22us/step - loss: 1.6864e-04
Epoch 568/800
799/799 [==============================] - 0s 19us/step - loss: 1.5614e-04
Epoch 569/800
799/799 [==============================] - 0s 21us/step - loss: 1.6137e-04
Epoch 570/800
799/799 [==============================] - 0s 21us/step - loss: 1.5678e-04
Epoch 571/800
799/799 [==============================] - 0s 20us/step - loss: 1.5265e-04
Epoch 572/800
799/799 [==============================] - 0s 19us/step - loss: 1.5027e-04
Epoch 573/800
799/799 [============

799/799 [==============================] - 0s 18us/step - loss: 9.8942e-05
Epoch 655/800
799/799 [==============================] - 0s 19us/step - loss: 9.6058e-05
Epoch 656/800
799/799 [==============================] - 0s 19us/step - loss: 9.6202e-05
Epoch 657/800
799/799 [==============================] - 0s 21us/step - loss: 1.0745e-04
Epoch 658/800
799/799 [==============================] - 0s 20us/step - loss: 9.9580e-05
Epoch 659/800
799/799 [==============================] - 0s 18us/step - loss: 1.0027e-04
Epoch 660/800
799/799 [==============================] - 0s 18us/step - loss: 1.0235e-04
Epoch 661/800
799/799 [==============================] - 0s 22us/step - loss: 1.0558e-04
Epoch 662/800
799/799 [==============================] - 0s 20us/step - loss: 1.0628e-04
Epoch 663/800
799/799 [==============================] - 0s 17us/step - loss: 9.5783e-05
Epoch 664/800
799/799 [==============================] - 0s 17us/step - loss: 1.0545e-04
Epoch 665/800
799/799 [============

799/799 [==============================] - 0s 14us/step - loss: 7.8942e-05
Epoch 747/800
799/799 [==============================] - 0s 14us/step - loss: 8.4701e-05
Epoch 748/800
799/799 [==============================] - 0s 14us/step - loss: 8.3468e-05
Epoch 749/800
799/799 [==============================] - 0s 15us/step - loss: 7.2958e-05
Epoch 750/800
799/799 [==============================] - 0s 13us/step - loss: 7.2707e-05
Epoch 751/800
799/799 [==============================] - 0s 16us/step - loss: 6.4247e-05
Epoch 752/800
799/799 [==============================] - 0s 18us/step - loss: 6.2371e-05
Epoch 753/800
799/799 [==============================] - 0s 13us/step - loss: 6.4760e-05
Epoch 754/800
799/799 [==============================] - 0s 12us/step - loss: 6.5142e-05
Epoch 755/800
799/799 [==============================] - 0s 13us/step - loss: 6.1735e-05
Epoch 756/800
799/799 [==============================] - 0s 14us/step - loss: 6.5369e-05
Epoch 757/800
799/799 [============

After the training is completed we can evaluate how good it is. To do this we can compute the residuals or the square root of the sum of the squared difference between the true value and the one predicted by the NN. We will also plot the true function and the predicted one in order to have a graphical representation of the goodness of our training.
To have a numerical estimate of the agreement it has been computed also the *mean squared error* defined as:

$MSE = \frac{\sum_{i=1}^n{\big(\frac{x_{i}^{pred} - x_i^{truth}}{x_i^{truth}}\big)^2}}{n}$

A *perfect* prediction would lead to $MSE=0$ so the lower this number the better the agreement. 

In [12]:
from sklearn.metrics import mean_squared_error

# report model error computing the mean squared error
print('MSE: %.3f' % mean_squared_error(trainer.y, trainer.predictions))

MSE: 0.016


To get an idea of what it is going on in the picture below are shown the actual function we want to approximate and different predictions of our NN obtained with four epoch numbers (5, 100, 800, 5000).

<img src="training_vs_epoch.png">

It is clear how the agreement improves with higher number of epochs which means that the NN has more opportunities to adapt the weights and reduce the loss (or error or distance) to the target values. Even in the case of 5000 epochs zooming in you could see discrepancies not visible at the scale of the plot. Clearly increasing further the number of epochs may lead to overfitting.

### Exercise 8.1

In order to see how different parameter choices affect the training (both looking at a plot like the one before and the the $MSE$) try to:

* reduce the number of points used in the training (change the step from 0.1 to 1 or to 0.01 in $\tt{x = arange(-50, 51, 0.1))}$, expect worse results with less points;
* change the number of nodes per layer;
* change the activation function from 'sigmoid' to 'relu';
* change the number of epochs, this is the number of times the neural network will process the sample data to improve the training; setting verbose to 1 will show the progress with an estimate of the goodness of the training after each epoch; expect worse training with less epoch.


### Black-Scholes call options

The first financial application of a NN concerns the pricing of european call options: essentially we will create a neural network capable of approximate the famous Black-Scholes pricing formula

$$ P_\textrm{call} = F_\textrm{BS}(K, r, \sigma, ttm)$$

Like before we are going to generate the training sample this time made of a grid of volatility-rate pairs $(\sigma, r)$ (for simplicity we are going to set moneyness and time to maturity to 1). The truth values are the price of a call computed using the pricing function in the $\tt{finmarkets.py}$ library with the corresponding inputs.

In [72]:
import numpy as np
from finmarkets import call

data = []
rates = np.arange(0.01, 0.11, 0.001)
sigmas = np.arange(0.1, 0.6, 0.005)

for r in rates:
    for sigma in sigmas:
        call_price = call(1, r, sigma, 1)
        data.append([r, sigma, call_price])
        
# we transform the list to a numpy array just because 
# an array it is more convenient to use later
data = np.array(data)

Since it takes some time to generate data samples, it is always advisable to save them in a file since we may need to load it many times during the NN development.
This can be done with $\tt{pandas}$.

In [73]:
import pandas as pd

df = pd.DataFrame()

df['vol'] = data[:, 1]
df['rate'] = data[:, 0]
df['price'] = data[:, 2]

df.to_csv("bs_training_sample.csv")

Following the previous example we will use the $\tt{FinNN}$ utility class to develop the NN and also we will *normalize* data to get better results.
**Beware that this time we have TWO input parameters (rate and volatilty)** and not just one.

Furthermore we will also split the generated sample into a training and a testing part so that we could later check for overfitting by comparing the perfomance in the two cases.

In [74]:
# first load back data
import pandas as pd
from finnn import FinNN

data =  pd.read_csv("bs_training_sample.csv")

x = data.iloc[:, 1:3].values
y = data.iloc[:, 3].values

In [75]:
trainer = FinNN()

trainer.setData(x, y, 0.20)
# the last parameter tells the class to split the original
# sample in training (80%) and testing parts (20%)
trainer.normalize()

# define the NN architecture
trainer.addInputLayer(2, 20, 'tanh')
trainer.addHiddenLayer(8, 'tanh')
trainer.addOutputLayer(1)
        
# define loss and optimizer algorithms
trainer.compileModel('mse', 'adam')
    
# run the training
# this time we are using many more epochs 
# and a larger batch_size
trainer.fit(2000, 500, 1)

# here we compare the performance 
# on the training and test sample
trainer.evaluate()

# when the training takes some time it is useful
# to save the model weights in a file to use it later on
trainer.saveModel()

Epoch 1/2000
8000/8000 [==============================] - 0s 15us/step - loss: 0.0101
Epoch 2/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.0039
Epoch 3/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.0022
Epoch 4/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.0019
Epoch 5/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.0016
Epoch 6/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.0014
Epoch 7/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.0012
Epoch 8/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.0010
Epoch 9/2000
8000/8000 [==============================] - 0s 4us/step - loss: 8.7041e-04
Epoch 10/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.3856e-04
Epoch 11/2000
8000/8000 [==============================] - 0s 4us/step - loss: 6.2612e-04
Epoch 12/2000
8000/8000 [=========================

8000/8000 [==============================] - 0s 5us/step - loss: 2.2831e-05
Epoch 93/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.2991e-05
Epoch 94/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.2398e-05
Epoch 95/2000
8000/8000 [==============================] - 0s 5us/step - loss: 2.1705e-05
Epoch 96/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.1626e-05
Epoch 97/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.1303e-05
Epoch 98/2000
8000/8000 [==============================] - 0s 5us/step - loss: 2.1023e-05
Epoch 99/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.1495e-05
Epoch 100/2000
8000/8000 [==============================] - 0s 5us/step - loss: 2.1253e-05
Epoch 101/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.0349e-05
Epoch 102/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.9891e-05
Epoch 103/2000
8000/8

8000/8000 [==============================] - 0s 3us/step - loss: 9.0018e-06
Epoch 183/2000
8000/8000 [==============================] - 0s 3us/step - loss: 9.5936e-06
Epoch 184/2000
8000/8000 [==============================] - 0s 3us/step - loss: 9.0749e-06
Epoch 185/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.2287e-06
Epoch 186/2000
8000/8000 [==============================] - 0s 4us/step - loss: 8.1846e-06
Epoch 187/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.1622e-06
Epoch 188/2000
8000/8000 [==============================] - 0s 4us/step - loss: 8.1942e-06
Epoch 189/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.9931e-06
Epoch 190/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.5685e-06
Epoch 191/2000
8000/8000 [==============================] - 0s 3us/step - loss: 9.5271e-06
Epoch 192/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.2761e-06
Epoch 193/2000

8000/8000 [==============================] - 0s 4us/step - loss: 5.6128e-06
Epoch 273/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.3863e-06
Epoch 274/2000
8000/8000 [==============================] - 0s 4us/step - loss: 6.4550e-06
Epoch 275/2000
8000/8000 [==============================] - 0s 4us/step - loss: 5.8785e-06
Epoch 276/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.0330e-06
Epoch 277/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.4479e-06
Epoch 278/2000
8000/8000 [==============================] - 0s 4us/step - loss: 5.0534e-06
Epoch 279/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.8934e-06
Epoch 280/2000
8000/8000 [==============================] - 0s 4us/step - loss: 5.0836e-06
Epoch 281/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.0109e-06
Epoch 282/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.8323e-06
Epoch 283/2000

8000/8000 [==============================] - 0s 5us/step - loss: 3.8296e-06
Epoch 362/2000
8000/8000 [==============================] - 0s 4us/step - loss: 4.5066e-06
Epoch 363/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.1195e-06
Epoch 364/2000
8000/8000 [==============================] - 0s 4us/step - loss: 3.7217e-06
Epoch 365/2000
8000/8000 [==============================] - 0s 4us/step - loss: 4.6873e-06
Epoch 366/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.0647e-06
Epoch 367/2000
8000/8000 [==============================] - 0s 4us/step - loss: 3.5706e-06
Epoch 368/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.5701e-06
Epoch 369/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.7448e-06
Epoch 370/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.7875e-06
Epoch 371/2000
8000/8000 [==============================] - 0s 4us/step - loss: 4.1628e-06
Epoch 372/2000

...

8000/8000 [==============================] - 0s 3us/step - loss: 1.2892e-06
Epoch 1609/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.5810e-06
Epoch 1610/2000
8000/8000 [==============================] - 0s 4us/step - loss: 5.4339e-07
Epoch 1611/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.8616e-07
Epoch 1612/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.1264e-07
Epoch 1613/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.6692e-06
Epoch 1614/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.9865e-06
Epoch 1615/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.5862e-06
Epoch 1616/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.2101e-07
Epoch 1617/2000
8000/8000 [==============================] - 0s 4us/step - loss: 8.2371e-07
Epoch 1618/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.1212e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 5.9793e-07
Epoch 1698/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.8294e-07
Epoch 1699/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.3968e-07
Epoch 1700/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.5428e-07
Epoch 1701/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4508e-06
Epoch 1702/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4574e-06
Epoch 1703/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.1340e-06
Epoch 1704/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3233e-06
Epoch 1705/2000
8000/8000 [==============================] - 0s 3us/step - loss: 9.5842e-07
Epoch 1706/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.2987e-06
Epoch 1707/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.0679e-07
Epoc

8000/8000 [==============================] - 0s 4us/step - loss: 6.2416e-07
Epoch 1787/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2910e-06
Epoch 1788/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.6399e-06
Epoch 1789/2000
8000/8000 [==============================] - 0s 3us/step - loss: 9.0456e-07
Epoch 1790/2000
8000/8000 [==============================] - 0s 4us/step - loss: 4.4541e-07
Epoch 1791/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.5573e-07
Epoch 1792/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.0598e-07
Epoch 1793/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.1542e-06
Epoch 1794/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.5946e-06
Epoch 1795/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.8279e-07
Epoch 1796/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.0358e-07
Epoc

8000/8000 [==============================] - 0s 4us/step - loss: 3.6477e-07
Epoch 1876/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.1760e-06
Epoch 1877/2000
8000/8000 [==============================] - 0s 4us/step - loss: 5.9153e-07
Epoch 1878/2000
8000/8000 [==============================] - 0s 4us/step - loss: 9.2046e-07
Epoch 1879/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.2955e-07
Epoch 1880/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.1814e-06
Epoch 1881/2000
8000/8000 [==============================] - 0s 4us/step - loss: 6.7743e-07
Epoch 1882/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4644e-06
Epoch 1883/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.1044e-07
Epoch 1884/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.6352e-07
Epoch 1885/2000
8000/8000 [==============================] - 0s 3us/step - loss: 8.5191e-07
Epoc

8000/8000 [==============================] - 0s 4us/step - loss: 4.2875e-07
Epoch 1965/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.4099e-07
Epoch 1966/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1848e-06
Epoch 1967/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.5364e-06
Epoch 1968/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.5593e-06
Epoch 1969/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.0017e-07
Epoch 1970/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.0721e-07
Epoch 1971/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.0647e-07
Epoch 1972/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.0251e-07
Epoch 1973/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.9751e-07
Epoch 1974/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.3399e-07
Epoc

As you can see the training and test samples give roughly the same $MSE$ value so we are reasonably sure that there hasn't been *overfitting*.
After the training is completed again we can evaluate graphically how good it is.
<img src="vol_rate.png">
We can also compare the prediction in a practical case; let's say we want to know the price of a call (with moneyness 1 and time to maturity 1 year) when the interest rate is 0.015 and the volatility 0.234:

In [78]:
import numpy as np
from finmarkets import call

# here we load the trained model
trainer.loadModel('test')

# this is our input vector
rv = np.array([[0.234, 0.015]])
# here we compare the predection with the BS call price                 
print ('{} => {:.4f} (expected {:.4f})'.format(rv.tolist(), 
                                        trainer.predict(rv)[0][0], 
                                        call(1, rv[0][1], rv[0][0], 1)))

[[0.234, 0.015]] => 0.0999 (expected 0.1001)


It is very import to remeber that a **NN cannot estrapolate**. Indeed if you try to predict the price of a call from rate and volatility outside the training *phase space* (with values that aren't in the intervals used in the training), say $r = 0.22$ and $\sigma = 0.01$...

In [79]:
# this is our input vector
rv = np.array([[0.01, 0.22]])
                 
# here we compare the predection with the BS call price                 
print ('{} => {:.4f} (expected {:.4f})'.format(rv.tolist(), 
                                        trainer.predict(rv)[0][0], 
                                        call(1, rv[0][1], rv[0][0], 1)))

[[0.01, 0.22]] => 0.1709 (expected 0.1975)


## Model Calibration

The function approximation of a neural network can serve other scopes rather than predicting prices. An very useful application is indeed *model calibration* which consists of deriving parameters of a model directly from market values. This is especially convenient to estimate parameters (e.g. volatility) which are otherwise complicated to compute.

Assume we need to estimate the implied volatilty of a stock price in real time. If in the market are available call options with out stock as underlying we can exploit again the Black and Scholes formula. The idea is in fact to train a NN where the input is a list of price, moneyness, rate and time to maturity $(P_\textrm{call}, K, r, ttm)$ and the target output is the volatility derived from the inversion of the call option pricing formula

$$ \sigma = F^{-1}_\textrm{BS}(P_\textrm{call}, K, r, ttm)$$

We can than calibrate our model by predicting the stock volatility with the trained NN using as input the market price of the option and its characteristics.

### Historical vs. Implied Volatility

Historical volatility is the realized volatility of the underlying asset over a previous time period. It is determined by measuring the standard deviation of the underlying asset from the mean during that time period.

Standard deviation is a statistical measure of the variability of price changes from the mean price change. This estimate differs from the Black-Scholes method's implied volatility, as it is based on the actual volatility of the underlying asset. However, using historical volatility also has some drawbacks. Volatility shifts as markets go through different regimes. Thus, historical volatility may not be an accurate measure of future volatility. Implied volatility takes into account all of the information used by market participants to determine prices in the options market, instead of just past prices.

As an example we can reuse the training sample created before (again we are going to set $T=1$ and $K=1$). Clearly now $\tt{x}$ will be pairs of rate and price and $\tt{y}$ the volatility.

In [80]:
import pandas as pd
from finnn import FinNN

data =  pd.read_csv("bs_training_sample.csv")

x = data.iloc[:, 2:4].values
y = data.iloc[:, 1].values

[[0.01       0.04485236]
 [0.01       0.04682511]
 [0.01       0.04879844]
 ...
 [0.109      0.27351542]
 [0.109      0.27529369]
 [0.109      0.27707117]]
[0.1   0.105 0.11  ... 0.585 0.59  0.595]


In [81]:
trainer = FinNN()

trainer.setData(x, y, 0.20)
trainer.normalize()

trainer.addInputLayer(2, 20, 'tanh')
trainer.addHiddenLayer(8, 'tanh')
trainer.addOutputLayer(1)

trainer.compileModel('mse', 'adam')
    
trainer.fit(2000, 500, 1)

trainer.evaluate()

trainer.saveModel("calibration")

Epoch 1/2000
8000/8000 [==============================] - 0s 15us/step - loss: 1.7378
Epoch 2/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.7167
Epoch 3/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.3199
Epoch 4/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.1917
Epoch 5/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.1302
Epoch 6/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.0928
Epoch 7/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.0708
Epoch 8/2000
8000/8000 [==============================] - 0s 6us/step - loss: 0.0575
Epoch 9/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.0482
Epoch 10/2000
8000/8000 [==============================] - 0s 5us/step - loss: 0.0409
Epoch 11/2000
8000/8000 [==============================] - 0s 4us/step - loss: 0.0348
Epoch 12/2000
8000/8000 [==============================] - 0s 

8000/8000 [==============================] - 0s 3us/step - loss: 7.3713e-04
Epoch 96/2000
8000/8000 [==============================] - 0s 3us/step - loss: 7.0882e-04
Epoch 97/2000
8000/8000 [==============================] - 0s 4us/step - loss: 6.8166e-04
Epoch 98/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.5536e-04
Epoch 99/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.3084e-04
Epoch 100/2000
8000/8000 [==============================] - 0s 3us/step - loss: 6.0714e-04
Epoch 101/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.8357e-04
Epoch 102/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.6213e-04
Epoch 103/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.4183e-04
Epoch 104/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.2200e-04
Epoch 105/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.0290e-04
Epoch 106/2000
800

8000/8000 [==============================] - 0s 3us/step - loss: 1.3882e-04
Epoch 185/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3710e-04
Epoch 186/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3462e-04
Epoch 187/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3355e-04
Epoch 188/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3245e-04
Epoch 189/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3092e-04
Epoch 190/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.2994e-04
Epoch 191/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.2806e-04
Epoch 192/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.2687e-04
Epoch 193/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.2585e-04
Epoch 194/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.2435e-04
Epoch 195/2000

8000/8000 [==============================] - 0s 3us/step - loss: 6.0285e-05
Epoch 275/2000
8000/8000 [==============================] - 0s 4us/step - loss: 5.9637e-05
Epoch 276/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.9280e-05
Epoch 277/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.8889e-05
Epoch 278/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.8423e-05
Epoch 279/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.7925e-05
Epoch 280/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.7650e-05
Epoch 281/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.7399e-05
Epoch 282/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.6861e-05
Epoch 283/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.6442e-05
Epoch 284/2000
8000/8000 [==============================] - 0s 3us/step - loss: 5.5747e-05
Epoch 285/2000

8000/8000 [==============================] - 0s 4us/step - loss: 3.2703e-05
Epoch 365/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.3153e-05
Epoch 366/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.2200e-05
Epoch 367/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.1766e-05
Epoch 368/2000
8000/8000 [==============================] - 0s 4us/step - loss: 3.1663e-05
Epoch 369/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.1962e-05
Epoch 370/2000
8000/8000 [==============================] - 0s 4us/step - loss: 3.1773e-05
Epoch 371/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.1751e-05
Epoch 372/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.1375e-05
Epoch 373/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.1379e-05
Epoch 374/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.0492e-05
Epoch 375/2000

...

8000/8000 [==============================] - 0s 3us/step - loss: 2.6348e-06
Epoch 1610/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.1607e-06
Epoch 1611/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.8340e-06
Epoch 1612/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.7167e-06
Epoch 1613/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.9907e-06
Epoch 1614/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.9712e-06
Epoch 1615/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.5354e-06
Epoch 1616/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.4701e-06
Epoch 1617/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.2700e-06
Epoch 1618/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.5971e-06
Epoch 1619/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.9180e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 1.7170e-06
Epoch 1698/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.6077e-06
Epoch 1699/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4629e-06
Epoch 1700/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.4485e-06
Epoch 1701/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.9112e-06
Epoch 1702/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.8669e-06
Epoch 1703/2000
8000/8000 [==============================] - ETA: 0s - loss: 3.0372e-0 - 0s 3us/step - loss: 1.9405e-06
Epoch 1704/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.6333e-06
Epoch 1705/2000
8000/8000 [==============================] - ETA: 0s - loss: 5.5605e-0 - 0s 3us/step - loss: 2.6018e-06
Epoch 1706/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.6248e-06
Epoch 1707/2000
8000/8000 [=============

8000/8000 [==============================] - 0s 3us/step - loss: 2.8821e-06
Epoch 1786/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.0628e-06
Epoch 1787/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.6586e-06
Epoch 1788/2000
8000/8000 [==============================] - 0s 3us/step - loss: 4.5809e-06
Epoch 1789/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.0552e-06
Epoch 1790/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.1566e-06
Epoch 1791/2000
8000/8000 [==============================] - 0s 4us/step - loss: 2.8627e-06
Epoch 1792/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.5818e-06
Epoch 1793/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.5144e-06
Epoch 1794/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.3903e-06
Epoch 1795/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4386e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 1.6473e-06
Epoch 1875/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.6676e-06
Epoch 1876/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.8983e-06
Epoch 1877/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.8433e-06
Epoch 1878/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.8231e-06
Epoch 1879/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.4245e-06
Epoch 1880/2000
8000/8000 [==============================] - 0s 4us/step - loss: 4.0149e-06
Epoch 1881/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.2586e-06
Epoch 1882/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.6659e-06
Epoch 1883/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.3194e-06
Epoch 1884/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.2403e-06
Epoc

8000/8000 [==============================] - 0s 3us/step - loss: 1.7182e-06
Epoch 1964/2000
8000/8000 [==============================] - 0s 3us/step - loss: 3.0400e-06
Epoch 1965/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.4414e-06
Epoch 1966/2000
8000/8000 [==============================] - 0s 4us/step - loss: 1.9638e-06
Epoch 1967/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4916e-06
Epoch 1968/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.4187e-06
Epoch 1969/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.9634e-06
Epoch 1970/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.4395e-06
Epoch 1971/2000
8000/8000 [==============================] - 0s 3us/step - loss: 1.9906e-06
Epoch 1972/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.6823e-06
Epoch 1973/2000
8000/8000 [==============================] - 0s 3us/step - loss: 2.0453e-06
Epoc

Provided our training includes the correct range of market prices of our call we can quickly and easily estimate the volatility, for example if the risk-free rate is 2% and the current price is 0.15 (remember that we are using the BS formula in terms of moneyness, strike divided by underlying price).

In [87]:
import numpy as np
from finmarkets import call

trainer.loadModel('calibration')

rv = np.array([[0.02, 0.15]])

print ('{} => {:.4f} (expected call price {:.4f})'.format(rv.tolist(), 
                                        trainer.predict(rv)[0][0], 
                                        call(1, 0.02, trainer.predict(rv)[0][0], 1)))

[[0.02, 0.15]] => 0.3554 (expected call price 0.1498)



## Neural net to recognize handwritten digits

We don't usually appreciate how tough a problem our visual system solve, maybe it is enough to consider that it involves 5 visual cortices containing 140 million neurons each. 
However the difficulties of visual pattern recognition become apparent if you attempt to write a computer program to recognize digits like those below

<img src="mnist_100_digits.png">

Simple intuition about how we recognize shapes (e.g. a 9 has a loop at the top, and a vertical stroke in the bottom right) turns out to be not so simple to express algorithmically. When you try to make such rules precise, you quickly get lost in a morass of exceptions and caveats and special cases so that it seems hopeless.

Neural networks approach the problem in a different way. The idea is to take a large number of handwritten digits and then develop a system which can learn from those. 

By increasing the number of training examples, the network can learn more and more about handwriting, and so improve its accuracy. So while it has been shown just 100 training digits above, we could certainly build a better handwriting recognizer by using thousands or even millions or billions of training examples (**as we have seen above neural nets are not capable of extrapolating results, hence it won't recongnize a digit written in some strange way not included in the training sample !!!**).

Let's try to implement a NN that is capable of recognizing handwritten digits.
To start we need to install another module, $\tt{mnist}$ which containes various predefined training samples.

Our program will be based on a slightly different kind of neural network, one type specifically designed for image/pattern recognition, the Convolutional Neural Network (CNN). We won't go in the details of its implementation since it is outside the scope of these lectures but it works essentially by applying on top of an image a series of filters (*convolutional layers*) that works as edge detectors. With them it classifies the images according to their relevant features.

Convolutional layers prove very effective, and stacking them allows to learn low-level features (e.g. lines) and high-order or more abstract features, like shapes or specific objects.

<img src="edges.jpg">

Another important difference with respect to the previous examples is that in this case we are going to solve a classification problem (contrary to before when we were trying to regress a sample or in other word to approximate a function). Indeed our NN output won't be a single number but rather a list containing the probabilties that an image belong to class on the classes.

In [61]:
import numpy as np
# contains our dataset for training
import mnist 

from finnn import FinNN

# load the training
train_images = mnist.train_images() # the actual images
train_labels = mnist.train_labels() # the truth (it is a 0, 1, 2...)

# 0 means do not split the sample in training and testing sets
# (MNIST has already dont it for us)
# the last parameter tells FinNN class that we are going to develop a CNN
trainer = FinNN("CNN2D")
trainer.setData(train_images, train_labels)
trainer.normalize()

# for technical reasons you need to expand axis
trainer.x = np.expand_dims(trainer.x, axis=3)

Next we define the CNN architecture.

In [62]:
# define our convolutional NN
# we decide to apply 8 filters to the images 
# each with 3x3 pixels size
# the input images have 28x28 pixels size instead
trainer.addConv2DLayer(8, 3, (28, 28, 1))

# the output is given by 10 neurons returning the 
# probability that image is in each class.
trainer.addCNNOutputLayer(10)
        
# adam is an algorithm to adjust the weights every cycle
# loss function compute the error between the prediction and the truth 
# metrics which error to use 
trainer.compileModel('categorical_crossentropy', 'adam')

trainer.fit(5)
#validation_data=(test_images, to_categorical(test_labels)))
    
trainer.saveModel('digit_training')

Epoch 1/5
59999/59999 [==============================] - 12s 199us/step - loss: 2.3481
Epoch 2/5
59999/59999 [==============================] - 12s 198us/step - loss: 0.3692
Epoch 3/5
59999/59999 [==============================] - 12s 197us/step - loss: 0.2321
Epoch 4/5
59999/59999 [==============================] - 12s 201us/step - loss: 0.1950
Epoch 5/5
59999/59999 [==============================] - 12s 200us/step - loss: 0.1856


### For the most curious students

If you look closely to the `finnn.py` module you will notice that I have cheated when
describing the CNN architecture. In particular I have not mentioned the `MaxPooling2D`
layer, so let's clarify its feature.

Convolutional layers in a convolutional neural network systematically apply learned filters to input images in order to create feature maps that summarize the presence of those features in the input.

A limitation of the feature map output of convolutional layers is that they record the precise position of features in the input. This means that small movements in the position of the feature in the input image will result in a different feature map. This can happen with re-cropping, rotation, shifting, and other minor changes to the input image.

Imagine a program that look for car plates in pictures taken by a speed radar, cars won't
be in the same position in the frame so there may be differences in the classification 
of similar (but not equal) pictures.

A common approach to address this problem from signal processing is called *down sampling*. This is where a lower resolution version of an input signal (e.g. the picture) is created that still contains the large or important structural elements, without the fine detail that may not be as useful to the task.

Down sampling can be achieved using a pooling layer.

Pooling involves selecting a pooling operation, much like a filter to be applied to feature maps. The size of the pooling operation or filter is smaller than the size of the feature map; specifically, it is almost always 2×2 pixels.
This means that the pooling layer will always reduce the size of each feature map by a factor of 2, e.g. each dimension is halved. For example, a pooling layer applied to a feature map of 6×6 (36 pixels) will result in an output pooled feature map of 3×3 (9 pixels).

The most common pooling operation are:
* Average Pooling: calculate the average value for each patch on the feature map;
* Maximum Pooling (or Max Pooling): calculate the maximum value for each patch of the feature map.


Now let's try to see how well our NN predicts MNIST testing digits.

In [63]:
import numpy as np
import mnist

trainer.loadModel('digit_training')

# testing with mnist test sample
test_images = mnist.test_images()
test_labels = mnist.test_labels()

test_images = np.expand_dims(test_images, axis=3)

predictions = trainer.predict(test_images[:10])
print ("Tesing on MNIST digits...")
print("Predicted: ", np.argmax(predictions, axis=1)) 
print("Truth:", test_labels[:10])

# this line returns the highest probability of the vector
print("highest prob.:", ["{:.6f}".format(p[np.argmax(p)]) for p in predictions])

Tesing on MNIST digits...
Predicted:  [7 2 1 0 4 1 4 9 5 9]
Truth: [7 2 1 0 4 1 4 9 5 9]
highest prob.: ['0.999999', '0.999928', '0.999889', '1.000000', '1.000000', '0.999999', '0.999977', '0.999983', '0.931519', '0.999535']


Since the last but one digit has lower probability let's check the returned list to see which other number have non-zero probability.

In [65]:
print("9th digit:", ["dig {}: {:.6f}".format(i, p) for i, p in enumerate(predictions[8])])

9th digit: ['dig 0: 0.000023', 'dig 1: 0.000000', 'dig 2: 0.000019', 'dig 3: 0.000000', 'dig 4: 0.000001', 'dig 5: 0.931519', 'dig 6: 0.047921', 'dig 7: 0.000000', 'dig 8: 0.020517', 'dig 9: 0.000000']


So the second ranked digit is a 6 (which can be confused with a five if the lower loop is almost closed). I am not sure how a 5 could be confused with a 8 though.

To see how well our NN behaves with different kind of digits we will try to check how it works with my calligraphy (as homework try to repeat the exercise using your own digit following the instructions given below).

* Open $\tt{paint}$ and create a 280x280 white square
* Change brush type and set the maximum size
* With the mouse draw a digit
* Finally save the file (e.g. five.png)

Before passing the image to the NN it has to be resized and this is done with an ad-hoc function ($\tt{transform\_image}$) which is in the $\tt{digit\_converter.py}$ module.

In [66]:
from digit_converter import transform_image

filenames = ['four.png', 'five.png']

for f in filenames:
    test_images = np.array(transform_image(f))
    test_images = np.expand_dims(test_images, axis=3)

    predict = trainer.predict(test_images)
    print ("\n")
    print ("Tesing on custom digits...")
    print ("Predicted: ", np.argmax(predict, axis=1))
    print("%:", ["{:.3f}".format(p[np.argmax(p)]) for p in predict])



Tesing on custom digits...
Predicted:  [4]
%: ['1.000']


Tesing on custom digits...
Predicted:  [5]
%: ['0.991']


Those the images I have checked:

<img src="four.png" width=80>
<img src="five.png" width=80>

## Model Calibration cont.

When the parameter(s) of our model we need to calibrate can be expressed as a function of three variables the CNN can be used. 

Consider again the Black and Scholes formula for the call options. Assume you need to calibrate the rate $r$ and the volatity $\sigma$. A convolutiona neural network can be trained fed with special images which represents $ttm, K$ and $P_\textrm{call}$.

A black-white image indeed can be interpreted as a map where each pixel is a pair ($ttm, K)$ and the pixel color, an integer between 0 (black) and 255 (white), represents $P_\textrm{call}$. As in the previous examples the neural network was classifing the pictures into digits, now it will assign them to classes identified by $r, \sigma$ pairs.

The creation of the training sample is a little more complicated now. For convenience we will use also a new format to save data image, $\tt{json}$. This will be done through the corresponding module simply using the functions $\tt{dump}$ and $\tt{load}$ to store and retrieve data.
The module $\tt{PIL}$ (pillow) is instead used to visualize the images.

In [ ]:
import numpy as np, json
import pandas as pd

# read rates and vols
data =  pd.read_csv("bs_training_sample.csv")

rates = data.iloc[:, 2].values
vols = data.iloc[:, 1].values

#k = np.arange(0.8, 1.2, (1.2-0.8)/20)
#ttm = np.arange(1, 5, 4/20)

## for each r, sigma pair
## generate a matrix of prices
#maximum = 0
#minimum = np.inf
#prices = []
#for v in vols:
#    for r in rates:
#        price =  np.zeros(shape=(20, 20))
#        for ik, kv in enumerate(k):
#            for it, t in enumerate(ttm):
#                price[ik, it] = call(kv, r, v, t)
#        prices.append(price)
#        # max and min are saved to 
#        # normalize our matrices
#        new_max = np.max(price)
#        new_min = np.min(price)
#        if new_max > maximum:
#            maximum = new_max
#        if new_min < minimum:
#            minimum = new_min
#            
# normalize each matrix in the range 0, 255
for ip, p in enumerate(prices):
    prices[ip] = np.interp(p, (minimum, maximum), (0, 255)) 

json.dump(prices, open("2d_label.json", "w"))

Below an example of the 20x20 images that have been created.

<img src="2d_training_images.png" width=200>

Then the training is similar to what has been done for the handwritten digits.

In [88]:
from finnn import FinNN
labels = []
for i in range(len(vols)):
    for j in range(len(rates)):
        labels.append((vols[i], rates[j]))            

images = np.array(json.load(open("2d.json")))

trainer = FinNN("CNN2D")
trainer.setData(images, labels, 0.2)
trainer.normalize()

trainer.addConv2DLayer(8, 3, (20, 20, 1), activatio='relu')
trainer.addFlatten()
trainer.addHiddenLayer(10, activation='relu')
trainer.addOutputLayer(2, activation='relu', bias_initializer='random_uniform')
trainer.compile('mse', 'adam')

trainer.fit(300, 32, verbose=1)
#model.save("2d.b5")
#eval1 = model.evaluate(x_train, z_train)
#print('Training: {}'.format(eval1))

#eval2 = model.evaluate(x_test, z_test)
#print('Test: {}'.format(eval2))


NameError: name 'vols' is not defined

### Technical Analysis

In finance, *technical analysis* is a security analysis discipline for forecasting the direction of prices through the study of past market data, primarily price and volume.
Essentially the analyst looks for particular patterns in the price time series that are *known* to develop in predictable ways to take profit of it.

<img src="H_and_s_top_new.jpg" width=400>
<img src="Triangle-ascending.jpg" width=400>

As you may imagine we will try to develop a CNN (like in the handwriting case) capable of classifying features in time series to be used in a technical analysis (this is much faster than having somebody looking at thousands of time series by eye...).

I have generated myself the training set simulating 21600 time series (1/3 with head and shoulder patter, 1/3 with triangle pattern and 1/3 with no pattern). *To make the training easier the features have been exagerated.*

<figure>
<img src="image_1.png" width=300>
<figcaption>No pattern</figcaption>
</figure>

<figure>
<img src="image_2.png" width=300>
<figcaption>Head and shoulder pattern</figcaption>
</figure>

<figure>
<img src="image_0.png" width=300>
<figcaption>Tringle pattern</figcaption>
</figure>

In [16]:
from finnn import FinNN
import pandas as pd
import numpy as np

train_labels = pd.read_csv("training_techana_labels.csv")
train_images = pd.read_csv("training_techana_images.csv")
#train_images = train_images[:3000]
#train_images = np.array(train_images)
train_images = np.expand_dims(train_images, axis=2)

trainer = FinNN("CNN1D")
trainer.setData(train_images, train_labels)

# define the CNN 
trainer.addConv1DInputLayer(80, 20, (101, 1))
trainer.addConv1DLayer(80, 15)
trainer.addMaxPooling1D(3)
trainer.addConv1DLayer(100, 10)
trainer.addConv1DLayer(100, 5)
trainer.addGlobalAveragePooling1D()
trainer.addDropout(0.5)
trainer.addCNNOutputLayer(3)

trainer.compileModel('categorical_crossentropy', 'adam')

# make the training
trainer.fit(80, 35)

trainer.saveModel('techana')

Epoch 1/80
21598/21598 [==============================] - 17s 774us/step - loss: 0.8423
Epoch 2/80
21598/21598 [==============================] - 16s 738us/step - loss: 0.6163
Epoch 3/80
21598/21598 [==============================] - 16s 748us/step - loss: 0.5836
Epoch 4/80
21598/21598 [==============================] - 16s 738us/step - loss: 0.5432
Epoch 5/80
21598/21598 [==============================] - 16s 730us/step - loss: 0.5513
Epoch 6/80
21598/21598 [==============================] - 15s 714us/step - loss: 0.5018
Epoch 7/80
21598/21598 [==============================] - 16s 721us/step - loss: 0.4953
Epoch 8/80
21598/21598 [==============================] - 16s 734us/step - loss: 0.4842
Epoch 9/80
21598/21598 [==============================] - 16s 730us/step - loss: 0.4737
Epoch 10/80
21598/21598 [==============================] - 16s 744us/step - loss: 0.4622
Epoch 11/80
21598/21598 [==============================] - 16s 738us/step - loss: 0.4431
Epoch 12/80
21598/21598 [=====

### Again for the most curious students

Large neural nets trained on relatively small datasets can overfit the training data.

This has the effect of the model learning the statistical noise in the training data, which results in poor performance when the model is evaluated on new data, e.g. a test dataset. 

One approach to reduce overfitting is to fit all possible different neural networks on the same dataset and to average the predictions from each model. This is not feasible in practice, and can be approximated using a small collection of different models, called an ensemble.
A problem even with the ensemble approximation is that it requires multiple models to be fit and stored, which can be a challenge if the models are large, requiring days or weeks to train and tune.

*Dropout* is a regularization method that approximates training a large number of neural networks with different architectures in parallel.

During training, some number of layer outputs are randomly ignored or *dropped out*. This has the effect of making the layer look-like and be treated-like a layer with a different number of nodes and connectivity to the prior layer. In effect, each update to a layer during training is performed with a different “view” of the configured layer.

Even if the it may seems counterintuitive (better training when switching off nodes) indeed dropout breaks-up situations where network layers co-adapt to correct mistakes from prior layers, in turn making the model more robust.

To test the perfomance I wanted to simulate a real case scenario where the time series are analyzed in real-time in order to predict as soon as possible a particular pattern and take advantage of the prediction.

TO do so I have created a longer time series and passed as input to the CNN sliding time windows to simulate the evolution of the time series. The goal was to check when the neural network was capable of predicting the incoming pattern.

<img src="closing_price.gif">

In [17]:
import numpy as np
import csv
from keras.models import Sequential, load_model
from keras.utils import to_categorical

test_images = pd.read_csv("testing_techana_frames.csv")
test_images = np.expand_dims(test_images, axis=2)

trainer.loadModel('techana')

predictions = trainer.predict(test_images)
for i in range(len(predictions)):
    print (np.argmax(predictions[i]), ["{:.2f}".format(p) for p in predictions[i]])

0 ['0.71', '0.00', '0.29']
0 ['0.66', '0.00', '0.34']
0 ['0.82', '0.00', '0.18']
0 ['0.91', '0.00', '0.09']
2 ['0.40', '0.06', '0.54']
1 ['0.00', '1.00', '0.00']
1 ['0.00', '1.00', '0.00']
1 ['0.00', '1.00', '0.00']
1 ['0.00', '1.00', '0.00']


So at the 6th sample the CNN start recognizing the *head and shoulder* pattern in the price evolution.

## Exercises

### Exercise 8.3
Taking as example the pricing NN trained on call, try to price put options.